## Data Preprocessing for Lending Club Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   
from pathlib import Path
import sys
repo_root = Path.cwd().resolve().parent   # repo root when notebook is in 'notebooks'
sys.path.append(str(repo_root))
from src.data_preprocessing import replace_by_mean, dataset_split_by_category, show_non_numeric_values, dataset_scaler


df = pd.read_csv("../data/raw/loans.csv")
print("First 5 rows of the dataset:")
print(df.head())

First 5 rows of the dataset:
   customer.id credit.policy             purpose  int.rate  installment  \
0        10001             1  debt_consolidation    0.1189       829.10   
1        10002             1         credit_card    0.1071       228.22   
2        10003             1  debt_consolidation    0.1357       366.86   
3        10004             1  debt_consolidation    0.1008       162.34   
4        10005             1         credit_card    0.1426       102.92   

   log.annual.inc    dti  fico  days.with.cr.line  revol.bal revol.util  \
0       11.350407  19.48   737        5639.958333    28854.0       52.1   
1       11.082143  14.29   707        2760.000000    33623.0       76.7   
2       10.373491  11.63   682        4710.000000     3511.0       25.6   
3       11.350407    8.1   712        2699.958333    33667.0       73.2   
4       11.299732  14.97   667        4066.000000     4740.0       39.5   

   inq.last.6mths delinq.2yrs pub.rec  not.fully.paid  
0            

#### Customer_id is an index_type column from source dataset, which has no value so can be dropped

In [ ]:
df.drop('customer.id', axis=1, inplace=True)

#### General overview shows columns of object type, which can mean mixed data inside and some nulls for numerical columns

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   object 
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9573 non-null   float64
 5   dti                9578 non-null   object 
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9549 non-null   float64
 8   revol.bal          9577 non-null   float64
 9   revol.util         9516 non-null   object 
 10  inq.last.6mths     9548 non-null   float64
 11  delinq.2yrs        9549 non-null   object 
 12  pub.rec            9549 non-null   object 
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(2), object(6)
memory usage: 1.0+ MB
None


#### Column credit.policy has string values, which can be replace by number equivalents

In [ ]:
print(df['credit.policy'].value_counts())

credit.policy
1       7709
0       1867
one        1
zero       1
Name: count, dtype: int64


In [ ]:
df['credit.policy'] = df['credit.policy'].map({'one': 1, 'zero': 0, '1': 1, '0': 0})
print(df['credit.policy'].value_counts())

credit.policy
1    7710
0    1868
Name: count, dtype: int64


#### Purpose column has 7 unique categories

In [ ]:
df['purpose'].unique()

array(['debt_consolidation', 'credit_card', 'all_other',
       'home_improvement', 'small_business', 'major_purchase',
       'educational'], dtype=object)

#### Column log.annual.inc has only 5 nulls, which can be replaces by mean values

In [ ]:
print(df['log.annual.inc'].isna().sum())

5


In [ ]:
replace_by_mean(df, 'log.annual.inc')


C:\Users\elena\Ellen\My_Projects\Lending-Club\src\data_preprocessing.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mean_value, inplace = True)


In [ ]:
print(df['log.annual.inc'].isna().sum())

0


#### Column dti has only one type of non-numeric vaue, which can be easily replace by number equivalent

In [ ]:
show_non_numeric_values(df, 'dti')

NAN values count: 0

Non-numeric values:
['six']

First 20 rows with non-numeric column:
['six']


In [ ]:
# Replace 'six' with '6.0' in dti column
df['dti'] = df['dti'].replace('six', '6.0')

# Convert column to numeric (handles any remaining string issues)
df['dti'] = pd.to_numeric(df['dti'], errors='coerce')

print(f"Data type after conversion: {df['dti'].dtype}")
print(f"Missing values (from conversion): {df['dti'].isna().sum()}")
print(f"\nFirst 20 values:\n{df['dti'].head(20)}")

Data type after conversion: float64
Missing values (from conversion): 0

First 20 values:
0     19.48
1     14.29
2     11.63
3      8.10
4     14.97
5     16.98
6      4.00
7     11.08
8     17.25
9     10.00
10    22.09
11     9.16
12    15.49
13     6.50
14     9.73
15    13.04
16     2.26
17     7.07
18     3.80
19     2.74
Name: dti, dtype: float64


#### More columns to replace nulls by mean values:

In [ ]:
replace_by_mean(df, 'days.with.cr.line')

In [ ]:
replace_by_mean(df, 'revol.bal')

#### More complex cleanup is required for revol.util column: str replaced by number, convertion to numeric, replacement of null values by mean

In [ ]:
show_non_numeric_values(df, 'revol.util')


NAN values count: 62

Non-numeric values:
[nan 'one']

First 20 rows with non-numeric column:
[nan, nan, nan, nan, nan, nan, nan, nan, 'one', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [ ]:
#1 . Replace 'one' with '1.0'
df['revol.util'] = df['revol.util'].replace('one', '1.0')

# 2. Convert to numeric (non-numeric values → NaN)
df['revol.util'] = pd.to_numeric(df['revol.util'], errors='coerce')

# 3. Fill NaN with mean
replace_by_mean(df, 'revol.util')

# 4. Verify
print(df['revol.util'].dtype)  # Should be float64
print(df['revol.util'].isna().sum())  # Should be 0


float64
0


#### More columns cleanup

In [ ]:
replace_by_mean(df, 'inq.last.6mths')
df['inq.last.6mths'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  8.,  7., 33.,  9., 18., 14.,
       15., 13., 12., 10., 19., 11., 16., 20., 27., 25., 28., 31., 24.,
       17., 32.])

In [ ]:
df['delinq.2yrs'].replace('yes', 1)    
df['delinq.2yrs'] = pd.to_numeric(df['delinq.2yrs'], errors='coerce')
replace_by_mean(df, 'delinq.2yrs')
df['delinq.2yrs'].value_counts()

delinq.2yrs
0.0     8463
1.0      827
2.0      192
3.0       65
4.0       19
5.0        6
6.0        2
13.0       1
7.0        1
8.0        1
11.0       1
Name: count, dtype: int64

In [ ]:
df['pub.rec'].value_counts()
df['pub.rec'].replace('no', 0)
df['pub.rec'] = pd.to_numeric(df['pub.rec'], errors='coerce')
replace_by_mean(df, 'pub.rec')
df['pub.rec'].unique()


C:\Users\elena\Ellen\My_Projects\Lending-Club\src\data_preprocessing.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mean_value, inplace = True)


array([0., 1., 2., 3., 4., 5.])

#### Exclude categorical columns before scaling

In [ ]:

df_numeric = df.drop(columns=['not.fully.paid', 'purpose'])

df_numeric.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   int.rate           9578 non-null   float64
 2   installment        9578 non-null   float64
 3   log.annual.inc     9578 non-null   float64
 4   dti                9578 non-null   float64
 5   fico               9578 non-null   int64  
 6   days.with.cr.line  9578 non-null   float64
 7   revol.bal          9578 non-null   float64
 8   revol.util         9578 non-null   float64
 9   inq.last.6mths     9578 non-null   float64
 10  delinq.2yrs        9578 non-null   float64
 11  pub.rec            9578 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 898.1 KB


#### One-hot encoding for purpose column and save result to file


In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories='auto', drop=None)
ohe_df = pd.DataFrame(ohe.fit_transform(df[['purpose']]).toarray(), columns=ohe.get_feature_names_out(['purpose']), index=df.index)
ohe_df.head()

,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
 scaled_features = dataset_scaler(df_numeric)

 feature_df = pd.DataFrame(scaled_features, columns=df_numeric.columns, index=df.index)
 df_scaled = pd.concat([feature_df, ohe_df, df['not.fully.paid']], axis=1)

In [ ]:
df_scaled.head()

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business,not.fully.paid
0,1.0,0.004023,0.879888,0.544762,0.650200,0.104167,0.312754,0.023898,0.077761,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
1,1.0,0.003217,0.229916,0.506334,0.476969,0.079167,0.147818,0.027848,0.114478,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.005171,0.379883,0.404820,0.388184,0.058333,0.259495,0.002908,0.038209,0.030303,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
3,1.0,0.002787,0.158653,0.544762,0.270360,0.083333,0.144379,0.027885,0.109254,0.030303,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,1.0,0.005642,0.094378,0.537503,0.499666,0.045833,0.222613,0.003926,0.058955,0.000000,0.076923,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
df_scaled.to_csv(f"../data/processed/loans_processed.csv", index=False)

#### Split to 7 datasets based on purpose column and save all 7 separately for further analysis

In [ ]:
df_scaled = pd.concat([feature_df,df['purpose'], df['not.fully.paid']], axis=1)
split_dfs = dataset_split_by_category(df_scaled, 'purpose')
# Store subsets in a dictionary instead of creating dynamic variable names
purpose_dfs = {}
for purpose, subset in split_dfs.items():
    print(f"Purpose: {purpose}, Number of records: {len(subset)}")
    #purpose_dfs[purpose] = subset
    temp_df = subset   
    safe_name = str(purpose).replace(' ', '_').replace('/', '_')
    temp_df.to_csv(f"../data/processed/loans_{safe_name}_processed.csv", index=False)
    purpose_dfs[safe_name] = temp_df
    
     

Purpose: debt_consolidation, Number of records: 3957
Purpose: credit_card, Number of records: 1262
Purpose: all_other, Number of records: 2331
Purpose: home_improvement, Number of records: 629
Purpose: small_business, Number of records: 619
Purpose: major_purchase, Number of records: 437
Purpose: educational, Number of records: 343


In [ ]:
purpose_dfs['debt_consolidation'].head()

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,purpose,not.fully.paid
0,1.0,0.004023,0.879888,0.544762,0.650200,0.104167,0.312754,0.023898,0.077761,0.000000,0.0,0.0,debt_consolidation,0
2,1.0,0.005171,0.379883,0.404820,0.388184,0.058333,0.259495,0.002908,0.038209,0.030303,0.0,0.0,debt_consolidation,0
3,1.0,0.002787,0.158653,0.544762,0.270360,0.083333,0.144379,0.027885,0.109254,0.030303,0.0,0.0,debt_consolidation,0
6,1.0,0.006120,0.192921,0.453657,0.133511,0.045833,0.171874,0.003180,0.114627,0.000000,0.0,0.2,debt_consolidation,1
9,1.0,0.004242,0.074042,0.380482,0.333778,0.079167,0.146102,0.004663,0.034328,0.030303,0.0,0.0,debt_consolidation,0


Conclusion: Dataset Cleanup is completed and two types of datasets are stored: one with one hot encoding, another is the group of datasets split by column Purpose for further experiements